In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib
import gc

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [19]:
from sklearn.preprocessing import StandardScaler

In [3]:
from tqdm import tqdm
import tensorflow as tf

from sklearn.metrics import average_precision_score, roc_auc_score, confusion_matrix

from scipy.special import softmax

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [5]:
!pip install feature_engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 2.6 MB/s eta 0:00:00


In [6]:
from feature_engine.imputation import MeanMedianImputer
from feature_engine.selection import DropConstantFeatures, SmartCorrelatedSelection, DropHighPSIFeatures

from sklearn.pipeline import Pipeline

In [7]:
!pip install pytorch-tabnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 573.4 kB/s eta 0:00:00


In [8]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.0 MB/s eta 0:00:00


In [9]:
from tensorflow_addons.activations import sparsemax

In [6]:
df_x = pd.read_pickle("/content/drive/MyDrive/Competition/norm_final_x_train.pkl")
df_y = pd.read_pickle("/content/drive/MyDrive/Competition/final_y_train (1).pkl")


In [11]:
final_test_df = pd.read_pickle("/content/drive/MyDrive/Competition/norm_final_test.pkl")

In [11]:
for col in df_x.columns:
    col_mean = df_x[col].mean()
    df_x[col].fillna(col_mean, inplace=True)

In [12]:
for col in final_test_df.columns:
    col_mean = final_test_df[col].mean()
    final_test_df[col].fillna(col_mean, inplace=True)

In [16]:
final_test_df

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,0.000000,0.003109,4,0.540814,0.022,0.378788,3,0.919708,2,0.159091,...,2,2,2,2,2,2226,0.444444,0.0,0.666667,0.75
1,0.000002,0.004769,4,0.188032,0.022,0.378788,3,0.919708,2,0.452273,...,2,2,2,2,2,2226,0.000000,0.0,0.666667,0.75
2,0.000004,0.016645,4,0.199759,0.948,0.378788,3,0.919708,2,0.845455,...,2,2,2,2,2,2226,0.555556,0.0,0.666667,0.75
3,0.000006,0.027744,4,0.574155,0.520,0.378788,3,0.481752,2,0.238636,...,2,2,2,2,2,2226,0.444444,0.0,0.666667,0.75
4,0.000008,0.006612,4,0.978213,0.704,0.378788,2,0.124088,2,0.372727,...,2,2,2,2,2,2226,0.444444,0.0,0.666667,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,0.999992,0.009216,0,0.737583,0.550,0.643939,2,0.905109,2,0.418182,...,2,2,2,2,2,2226,0.444444,0.0,0.444444,0.00
506687,0.999994,0.001183,0,0.123764,0.616,0.643939,2,0.905109,2,0.436015,...,0,0,1,0,1,107,0.555556,0.0,0.555556,0.00
506688,0.999996,0.004769,4,0.900207,0.780,0.378788,3,0.919708,2,0.515909,...,2,2,2,2,2,2226,0.555556,0.0,0.666667,0.75
506689,0.999998,0.019663,4,0.897908,0.832,0.378788,2,0.905109,2,0.175000,...,2,2,2,2,2,2226,0.555556,0.0,0.666667,0.75


In [ ]:
k = 0
for i in final_test_df.isnull().sum():
    print(i)
    k+=1

print(k)


In [17]:
df_x

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,0.000000,0.002137,4,0.743044,NaN,0.381679,2,0.306569,2,0.488636,...,2,2,2,2,2,1786,0.666667,0.75,0.666667,0.75
1,0.000002,0.000900,4,0.100885,0.608,0.381679,3,0.014599,2,0.511364,...,2,2,2,2,2,1786,0.444444,0.00,0.666667,0.75
2,0.000003,0.001840,4,0.210566,0.780,0.381679,4,0.481752,3,0.522727,...,2,2,2,2,2,1786,0.555556,0.00,0.666667,0.75
3,0.000005,0.001558,4,0.984824,0.934,0.381679,3,0.124088,3,0.854545,...,2,2,2,2,2,1786,1.000000,0.00,0.666667,0.75
4,0.000007,0.001558,1,0.201023,0.828,0.381679,3,0.014599,2,0.727273,...,1,0,1,1,1,954,0.444444,0.00,0.666667,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0.999993,0.001526,4,0.319039,NaN,0.381679,4,0.919708,3,0.390909,...,2,2,2,2,2,1786,0.666667,0.75,0.666667,0.75
590536,0.999995,0.001229,4,0.542883,0.250,0.381679,3,0.905109,3,0.236364,...,2,2,2,2,2,1786,0.444444,0.00,0.666667,0.75
590537,0.999997,0.000961,4,0.634456,0.990,0.381679,3,0.905109,3,0.297727,...,2,2,2,2,2,1786,0.444444,0.00,0.666667,0.75
590538,0.999998,0.003656,4,0.392389,0.762,0.381679,3,0.905109,3,0.652273,...,2,2,2,2,2,1786,0.000000,0.00,0.666667,0.75


In [27]:
X_train.to_pickle('/content/drive/MyDrive/Competition/X_train')
X_val.to_pickle('/content/drive/MyDrive/Competition/X_val')
y_train.to_pickle('/content/drive/MyDrive/Competition/y_train')
y_val.to_pickle('/content/drive/MyDrive/Competition/y_val')

In [13]:
final_test_df.to_pickle('/content/drive/MyDrive/Competition/main_test')

In [21]:
X_train, X_val, y_train, y_val = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [14]:
del final_test_df

In [15]:
gc.collect()

102

In [26]:
type(X_train)

pandas.core.frame.DataFrame

In [22]:
X_train.shape

(472432, 436)

In [13]:
X_train = pd.read_pickle('/content/drive/MyDrive/Competition/X_train')
X_val = pd.read_pickle('/content/drive/MyDrive/Competition/X_val')
y_train = pd.read_pickle('/content/drive/MyDrive/Competition/y_train')
y_val = pd.read_pickle('/content/drive/MyDrive/Competition/y_val')

In [14]:
X_test = pd.read_pickle('/content/drive/MyDrive/Competition/main_test')

In [23]:
print(f'X_train: {X_train.shape}')
print(f'X_val: {X_val.shape}')
print(f'Y_train: {y_train.shape}')
print(f'Y_val: {y_val.shape}')

X_train: (472432, 436)
X_val: (118108, 436)
Y_train: (472432,)
Y_val: (118108,)


In [24]:
del df_x
del df_y

In [25]:
gc.collect()

0

## **TF DATA**

This code defines a function prepare_tf_dataset to create TensorFlow datasets from input data (X, and optionally y for labels) for training, validation, or testing. The function performs the following steps:

**One-Hot Encoding for Labels (y):**

If y (labels) is provided, it converts the labels to one-hot encoded format using tf.one_hot with two classes.

**Creating TensorFlow Dataset:**

Uses tf.data.Dataset.from_tensor_slices to create a TensorFlow dataset.
If y is provided, it creates a dataset with pairs (X, y) where X is the feature data and y is the one-hot encoded labels.
If y is not provided, it creates a dataset only with X (feature data).

**Shuffling (if specified):**

If shuffle is set to True, shuffles the dataset using ds.shuffle.

**Batching:**

Batches the dataset into smaller chunks using ds.batch with a specified batch size (batch_size).
drop_remainder is used to drop the last batch if its size is less than batch_size.

**Prefetching:**

Uses ds.prefetch to prefetch data for better performance during training. The argument autotune is set to tf.data.experimental.AUTOTUNE for automatic optimization.

**Function Return:**

Returns the prepared TensorFlow dataset.

**Usage:**

The function is then used to create TensorFlow datasets for training (train_ds), validation (val_ds), and testing (test_ds) using provided data (X_train, y_train, X_val, y_val, X_test).

In [ ]:
def prepare_tf_dataset(
    X,
    batch_size,
    y = None,
    shuffle = False,
    drop_remainder = False,
):

    """
    Creates a TensorFlow dataset for training, validation, or testing.

    Parameters:
    - X: Feature data.
    - batch_size: Batch size for batching the dataset.
    - y: Labels (optional).
    - shuffle: Whether to shuffle the dataset (default: False).
    - drop_remainder: Whether to drop the last batch if its size is less than batch_size (default: False).

    Returns:
    - TensorFlow dataset prepared for training, validation, or testing.
    """

    size_of_dataset = len(X)
    if y is not None:
        y = tf.one_hot(y.astype(int), 2)
        ds = tf.data.Dataset.from_tensor_slices((np.array(X.astype(np.float32)), y))
    else:
        ds = tf.data.Dataset.from_tensor_slices(np.array(X.astype(np.float32)))
    if shuffle:
        ds = ds.shuffle(buffer_size=size_of_dataset)
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)

    autotune = tf.data.experimental.AUTOTUNE
    ds = ds.prefetch(autotune)
    return ds

train_ds = prepare_tf_dataset(X_train, 16000, y_train)
val_ds = prepare_tf_dataset(X_val, 16000, y_val)
test_ds = prepare_tf_dataset(X_test, 16000)

In [ ]:
print(train_ds)
print(val_ds)

# **Defining the TabNet Architecture**

In [22]:
def glu(x, n_units=None):
    """Generalized linear unit nonlinear activation."""
    return x[:, :n_units] * tf.nn.sigmoid(x[:, n_units:])

In [21]:
class FeatureBlock(tf.keras.Model):
    """
    Implementation of a FL->BN->GLU block
    """
    def __init__(
        self,
        feature_dim,
        apply_glu = True,
        bn_momentum = 0.9,
        fc = None,
        epsilon = 1e-5,
    ):

        """
        Initializes the FeatureBlock.

        Parameters:
        - feature_dim: Dimensionality of the features.
        - apply_glu: Whether to apply GLU activation (default: True).
        - bn_momentum: Batch normalization momentum.
        - fc: Dense layer for shared weights.
        - epsilon: Batch normalization epsilon.

        Returns:
        - None
        """
        super(FeatureBlock, self).__init__()
        self.apply_gpu = apply_glu
        self.feature_dim = feature_dim
        units = feature_dim * 2 if apply_glu else feature_dim # desired dimension gets multiplied by 2
                                                              # because GLU activation halves it
        self.fc = tf.keras.layers.Dense(units, use_bias=False) if fc is None else fc # shared layers can get re-used
        self.bn = tf.keras.layers.BatchNormalization(momentum=bn_momentum, epsilon=epsilon)


    def call(self, x, training = None):

        """
        Defines the forward pass of FeatureBlock.

        Parameters:
        - x: Input features.
        - training: Whether the model is in training mode.

        Returns:
        - Output of the FeatureBlock.
        """
        x = self.fc(x) # inputs passes through the FC layer
        x = self.bn(x, training=training) # FC layer output gets passed through the BN
        if self.apply_gpu:
            return glu(x, self.feature_dim) # GLU activation applied to BN output
        return x


class FeatureTransformer(tf.keras.Model):
    def __init__(
        self,
        feature_dim,
        fcs = [],
        n_total = 4,
        n_shared = 2,
        bn_momentum = 0.9,
    ):

        """
        Initializes the FeatureTransformer.

        Parameters:
        - feature_dim: Dimensionality of the features.
        - fcs: List of shared Dense layers.
        - n_total: Total number of FeatureBlocks.
        - n_shared: Number of shared FeatureBlocks.
        - bn_momentum: Batch normalization momentum.

        Returns:
        - None
        """
        super(FeatureTransformer, self).__init__()
        self.n_total, self.n_shared = n_total, n_shared

        kwrgs = {
            "feature_dim": feature_dim,
            "bn_momentum": bn_momentum,
        }

        self.blocks = []
        for n in range(n_total):
            if fcs and n < len(fcs):
                self.blocks.append(FeatureBlock(**kwrgs, fc=fcs[n])) # Building shared blocks by providing FC layers
            else:
                self.blocks.append(FeatureBlock(**kwrgs)) # Step dependent blocks without the shared FC layers

    def call(self, x, training = None):

        """
        Defines the forward pass of FeatureTransformer.

        Parameters:
        - x: Input features.
        - training: Whether the model is in training mode.

        Returns:
        - Output of the FeatureTransformer.
        """
        x = self.blocks[0](x, training=training)
        for n in range(1, self.n_total):
            # output from previous block gets multiplied by sqrt(0.5) and output of this block gets added
            x = x * tf.sqrt(0.5) + self.blocks[n](x, training=training)
        return x

    @property
    def shared_fcs(self):
        return [self.blocks[i].fc for i in range(self.n_shared)]

class AttentiveTransformer(tf.keras.Model):
    def __init__(self, feature_dim):
        super(AttentiveTransformer, self).__init__()
        self.block = FeatureBlock(
            feature_dim,
            apply_glu=False,
        )

    def call(self, x, prior_scales, training=None):
        x = self.block(x, training=training)
        return sparsemax(x * prior_scales)

class TabNet(tf.keras.Model):
    def __init__(
        self,
        num_features = 436,
        feature_dim = 128,
        output_dim = 128,
        n_step = 2,
        n_total = 4,
        n_shared = 2,
        relaxation_factor = 1.5,
        bn_epsilon = 1e-5,
        bn_momentum = 0.7,
        sparsity_coefficient = 1e-5
    ):

        """
        Initializes the AttentiveTransformer.

        Parameters:
        - feature_dim: Dimensionality of the features.

        Returns:
        - None
        """
        super(TabNet, self).__init__()
        self.output_dim, self.num_features = output_dim, num_features
        self.n_step, self.relaxation_factor = n_step, relaxation_factor
        self.sparsity_coefficient = sparsity_coefficient

        self.bn = tf.keras.layers.BatchNormalization(
            momentum=bn_momentum, epsilon=bn_epsilon
        )

        kargs = {
            "feature_dim": feature_dim + output_dim,
            "n_total": n_total,
            "n_shared": n_shared,
            "bn_momentum": bn_momentum
        }

        # first feature transformer block is built first to get the shared blocks
        self.feature_transforms = [FeatureTransformer(**kargs)]
        self.attentive_transforms = []

        # each step consists out of FT and AT
        for i in range(n_step):
            self.feature_transforms.append(
                FeatureTransformer(**kargs, fcs=self.feature_transforms[0].shared_fcs)
            )
            self.attentive_transforms.append(
                AttentiveTransformer(num_features)
            )

        # Final output layer
        self.head = tf.keras.layers.Dense(2, activation="softmax", use_bias=False)

    def call(self, features, training = None):

        """
        Defines the forward pass of AttentiveTransformer.

        Parameters:
        - x: Input features.
        - prior_scales: Prior scales for attention.
        - training: Whether the model is in training mode.

        Returns:
        - Output of the AttentiveTransformer.
        """

        bs = tf.shape(features)[0] # get batch shape
        out_agg = tf.zeros((bs, self.output_dim)) # empty array with outputs to fill
        prior_scales = tf.ones((bs, self.num_features)) # prior scales initialised as 1s
        importance = tf.zeros([bs, self.num_features]) # importances
        masks = []

        features = self.bn(features, training=training) # Batch Normalisation
        masked_features = features

        total_entropy = 0.0

        for step_i in range(self.n_step + 1):
            # (masked) features go through the FT
            x = self.feature_transforms[step_i](
                masked_features, training=training
            )

            # first FT is not used to generate output
            if step_i > 0:
                # first half of the FT output goes towards the decision
                out = tf.keras.activations.relu(x[:, : self.output_dim])
                out_agg += out
                scale_agg = tf.reduce_sum(out, axis=1, keepdims=True) / (self.n_step - 1)
                importance += mask_values * scale_agg


            # no need to build the features mask for the last step
            if step_i < self.n_step:
                # second half of the FT output goes as input to the AT
                x_for_mask = x[:, self.output_dim :]

                # apply AT with prior scales
                mask_values = self.attentive_transforms[step_i](
                    x_for_mask, prior_scales, training=training
                )

                # recalculate the prior scales
                prior_scales *= self.relaxation_factor - mask_values

                # multiply the second half of the FT output by the attention mask to enforce sparsity
                masked_features = tf.multiply(mask_values, features)

                # entropy is used to penalize the amount of sparsity in feature selection
                total_entropy += tf.reduce_mean(
                    tf.reduce_sum(
                        tf.multiply(-mask_values, tf.math.log(mask_values + 1e-15)),
                        axis=1,
                    )
                )

                # append mask values for later explainability
                masks.append(tf.expand_dims(tf.expand_dims(mask_values, 0), 3))

        #Per step selection masks
        self.selection_masks = masks

        # Final output
        final_output = self.head(out)

        # Add sparsity loss
        loss = total_entropy / (self.n_step-1)
        self.add_loss(self.sparsity_coefficient * loss)

        return final_output, importance

In [34]:
tabnet = TabNet(num_features = 436,
                output_dim = 128,
                feature_dim = 128,
                n_step = 2,
                relaxation_factor= 2,
                sparsity_coefficient=2.5e-07,
                n_shared = 2,
                bn_momentum = 0.93)


In [35]:
cbs = [tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=30, restore_best_weights=True
    )]

In [36]:
# Optimiser
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=10)

# Second loss in None because we also output the importances
loss = [tf.keras.losses.CategoricalCrossentropy(from_logits=False), None]

tabnet.compile(optimizer, loss=loss)

In [22]:
tabnet.fit(train_ds,
           epochs=100,
           validation_data=val_ds,
           callbacks=cbs,
           verbose=1,
          class_weight={
              0:1,
              1:20
          })

Epoch 1/100
30/30 [==============================] - 35s 354ms/step - loss: 1.0754 - output_1_loss: 1.0754 - val_loss: 0.5089 - val_output_1_loss: 0.5089
Epoch 2/100
30/30 [==============================] - 11s 357ms/step - loss: 0.8860 - output_1_loss: 0.8860 - val_loss: 0.4425 - val_output_1_loss: 0.4425
Epoch 3/100
30/30 [==============================] - 9s 287ms/step - loss: 0.8099 - output_1_loss: 0.8099 - val_loss: 0.3795 - val_output_1_loss: 0.3795
Epoch 4/100
30/30 [==============================] - 9s 290ms/step - loss: 0.7740 - output_1_loss: 0.7740 - val_loss: 0.3422 - val_output_1_loss: 0.3422
Epoch 5/100
30/30 [==============================] - 8s 277ms/step - loss: 0.7432 - output_1_loss: 0.7432 - val_loss: 0.3331 - val_output_1_loss: 0.3331
Epoch 6/100
30/30 [==============================] - 9s 280ms/step - loss: 0.7148 - output_1_loss: 0.7148 - val_loss: 0.3315 - val_output_1_loss: 0.3315
Epoch 7/100
30/30 [==============================] - 8s 277ms/step - loss: 0.700

In [ ]:
tabnet.save_weights("/content/drive/MyDrive/Competition/Models_check/tabnet_weights.h5")


In [24]:
from sklearn.metrics import roc_auc_score, average_precision_score

val_preds, val_imps = tabnet.predict(val_ds)

8/8 [==============================] - 2s 99ms/step


In [26]:
print('Test ROC AUC', np.round(roc_auc_score(y_val, val_preds[:, 1]), 4))
print('Test PR AUC', np.round(average_precision_score(y_val, val_preds[:, 1]), 4))

Test ROC AUC 0.9168
Test PR AUC 0.6232


____________________

_________________________